# ZFC
https://github.com/sneeuwballen/zipperposition/blob/master/examples/ho/Axioms/SET008%5E0.ax
: [BS+05] Benzmueller et al. (2005), Can a Higher-Order and a Fi
%          : [BS+08] Benzmueller et al. (2007), Combined Reasoning by Autom
%          : [Ben08] Benzmueller (2008), Email to Geoff Sutcliffe

quaife
isabelle zf https://isabelle.in.tum.de/dist/Isabelle2024/doc/logics-ZF.pdf

In [1]:

# Zf
Set = DeclareSort("Set")
elem = Function("elem", Set, Set, BoolSort())
A,B,C,x,y,z = Consts("A B C x y z", Set) 

ax_emp = trust(Exists([A],ForAll([x], ~elem(B,A))))
ax_pair = trust(ForAll([A,B], Exists([C], ForAll([x], elem(x, C) == elem(x,A) | elem(x,B)))))
ax_ext = trust(ForAll([A,B], ForAll([x], elem(x,A) == elem(x,B)) == (A == B)))
ax_union = trust(ForAll([A], Exists([B], ForAll([x], elem(x,B) == Exists([y], elem(x,y) & elem(y,A))))))
def ax_sep(B,P):
    A = FreshConst(Set)
    return trust(Exists([A], ForAll([x], elem(x,A) == elem(x,B) & P(x))))

# ordered pairs

# injectivity theorem

# conservative extension as a definition schema
def extend(hyps,x,P, prefix):
    infer(hyps, Exists([x], P))
    s = FreshConst(Set, prefix=prefix)
    return s, trust(P.substitute(s, x))

NameError: name 'DeclareSort' is not defined

So I stepped right onto a Lambda landmine. Don't use lambdas. Don't use existentials. You put yourself into a place where you are in z3's thrall. 
Or do use them to write definitions but quickly get something else to work with.

In [18]:
import knuckledragger as kd
from z3 import *
Set = DeclareSort("Set")
x,y,z,A,B,C = Consts("x y z A B C", Set)
Class = Set >> BoolSort()
P,Q,R = Consts("P Q R", Class)
elem = Function("elem", Set, Set,BoolSort())
Set.ext = kd.axiom(ForAll([A,B], (A == B) == ForAll([x], elem(x,A) == elem(x,B))))
emp = Const("emp", Set)
Set.emp = emp
emp_def = kd.axiom(ForAll([x], ~elem(x, Set.emp)))
emp.defn = emp_def


upair = Function("upair", Set, Set, Set)
upair_def = kd.axiom(ForAll([x,y,z], elem(z, upair(x,y)) == ((z == x) | (z == y))))

sep = Function("sep", Set, Class, Set)
sep_def = kd.axiom(ForAll([P,x,z], elem(z, sep(x,P)) == (elem(z,x) & P[z]), patterns=[elem(z,sep(x,P))]))


#biginter, biginter_def = define("biginter", [A], sep(A, ))


le = kd.notation.le.define([A,B], ForAll([x], Implies(elem(x,A), elem(x,B))))

le_eq = kd.lemma(ForAll([A,B], (A <= B) & (B <= A) == (A == B)), by=[le.defn, Set.ext])

# kuratowski
pair = kd.define("pair", [A,B], upair(upair(A,A),upair(A,B)))

bigunion = Function("bigunion", Set, Set)
# hmm. using a existnetial. I don't love that.
# The skolem is similar to choice, except it takes in the element and the big set. choice takes in big set and set in big set and gives element it chose.
bigunion_def = kd.axiom(ForAll([x,A], elem(x, bigunion(A)) == Exists([y], elem(x,y) & elem(y,A))))

union = kd.define("union", [A,B], bigunion(upair(A,B)))
union_elem = kd.lemma(ForAll([x,A,B], elem(x, union(A,B)) == (elem(x,A) | elem(x,B))), by=[union.defn, upair_def, bigunion_def])

union_comm = kd.lemma(ForAll([A,B], union(A,B) == union(B,A)), by=[union_elem, Set.ext])
union_idem = kd.lemma(ForAll([A], union(A,A) == A), by=[union_elem, Set.ext])

klass = kd.define("class", [A], Lambda([x], elem(x,A)))

elem_klass = kd.lemma(ForAll([A,x], klass(A)[x] == elem(x,A)), by=[klass.defn])

inter = kd.define("inter", [A,B], sep(A, klass(B)))

elem_inter = kd.lemma(ForAll([A,B,x], elem(x, inter(A,B)) == (elem(x,A) & elem(x,B))), by=[inter.defn, elem_klass, sep_def])

elem_inter2 = kd.lemma(ForAll([A,B,x], elem(x, inter(A,B)) == elem(x,A) & elem(x,B)), by=[inter.defn, elem_klass, sep_def])


LemmaError: ('lemma', ForAll([A, B], union(A, B) == union(B, A)), unknown)

In [14]:

# This one is finicky between z3 runs
# Isn't there some paper that says that if z3 has stocahstic runtimes, you should reformulate,
#inter_alt = lemma(ForAll([A,B], inter(A,B) == sep(B, klass(A))), by=[elem_inter, inter_def,  Set.ext], timeout=1000)

inter_comm = kd.lemma(ForAll([A,B], inter(A,B) == inter(B,A)), by=[elem_inter,  Set.ext], timeout=1000)
inter_alt_def = kd.lemma(ForAll([A,B], inter(A,B) == sep(B, klass(A))), by=[inter.defn,  inter_comm], timeout=1000)

inter_idem = kd.lemma(ForAll([A], inter(A,A) == A), by=[elem_inter, Set.ext])


# elem(x, inter(A,B)) == elem(x,A) & elem(x,B) == elem(x, inter(B,A)) == 

print(elem_inter)
print(elem_inter2)


LemmaError: ('lemma', ForAll([A, B], inter(A, B) == inter(B, A)), unknown)

In [2]:
def sepschema(A,P):
    B = FreshConst(Set)
    return B, kd.axiom(ForAll([x], elem(x, B) == (elem(x,A) & P[x])))
#fst, fst_def = define("fst", [A], sep(A, Lambda([x], elem(x, upair(A,A))))

inter,inter_def = kd.define("inter", [A,B], sep(A, Lambda([x], elem(x,B))))

l1 = kd.lemma(ForAll([A,B], inter(A,B) == sep(A, Lambda([x], elem(x,B)))), by=[inter_def])
#l2 = lemma(ForAll([A,B], sep(B, Lambda([x], elem(x,A)))), by=[inter_def, Set.ext, l1]
#lemma(ForAll([A,B], inter(A,B) == elem(x,A) & elem(x,B)), by=[inter_def, sep_def])
l3 = kd.lemma(ForAll([A,B,x], elem(x, sep(A, Lambda([x], elem(x, B)))) == elem(x,A) & elem(x,B)), by=[sep_def])
l4 = kd.lemma(ForAll([A,B,x], elem(x, sep(B, Lambda([x], elem(x, A)))) == elem(x,A) & elem(x,B)), by=[sep_def])
l6 = kd.lemma(ForAll([A,B,x], 
                  And(elem(x, sep(A, Lambda([x], elem(x,B)))) == (elem(x,A) & elem(x,B)),
                      elem(x, sep(B, Lambda([x], elem(x,A)))) == (elem(x,A) & elem(x,B)))), by=[l3, l4], dump=True)
#l5 = lemma(ForAll([A,B,x], elem(x, sep(A, Lambda([x], elem(x,B)))) == elem(x, sep(B, Lambda([x], elem(x,A))))), by=[l6])

#l2 = lemma(ForAll([x], ForAll([A,B], 
#                               Implies(inter(A,B) == sep(A, Lambda([x], elem(x,B))), 
#                                elem(x, inter(A,B)) == elem(x, sep(A, Lambda([x], elem(x, B))))))), by=[inter_def, l1])
#l4 = lemma(ForAll([A,B], elem(x, sep(A, Lambda([x], elem(x, B))) == elem(x, A) & elem(x, B)), by=[sep_def, l1]
#l2 = lemma(ForAll([A,B], elem(x, inter(A,B)) == elem(x, A) & elem(x, B)), by=[inter_def, l1, l3])
#l2 = lemma(ForAll([A,B,x], elem(x, sep(A, Lambda([x], elem(x, B)))) == elem(x, A) | elem(x, B)), by=[sep_def])
#lemma(ForAll([A,B,x], elem(x, inter(A,B)) == elem(x,A) | elem(x,B)), by=[le_eq, inter_def, sep_def, l1])

sep_true = kd.lemma(ForAll([A], sep(A,Lambda([x], BoolVal(True))) == A), by=[sep_def, Set.ext])
sep_false = kd.lemma(ForAll([A], sep(A,Lambda([x], BoolVal(False))) == Set.emp), by=[sep_def, Set.ext, emp_def])

(declare-sort Set 0)
(declare-fun elem (Set Set) Bool)
(declare-fun sep (Set (Array Set Bool)) Set)
(declare-fun knuckledragger_goal () Bool)
(assert (forall ((A Set) (B Set) (x Set))
  (let ((a!1 (elem x (sep A (lambda ((x!1 Set)) (elem x!1 B))))))
    (= a!1 (and (elem x A) (elem x B))))))
(assert (forall ((A Set) (B Set) (x Set))
  (let ((a!1 (elem x (sep B (lambda ((x!1 Set)) (elem x!1 A))))))
    (= a!1 (and (elem x A) (elem x B))))))
(assert (let ((a!1 (forall ((A Set) (B Set) (x Set))
             (let ((a!1 (elem x (sep A (lambda ((x!1 Set)) (elem x!1 B)))))
                   (a!2 (elem x (sep B (lambda ((x!1 Set)) (elem x!1 A))))))
               (and (= a!1 (and (elem x A) (elem x B)))
                    (= a!2 (and (elem x A) (elem x B))))))))
  (=> knuckledragger_goal (not a!1))))



LemmaError: ('lemma', ForAll([A, B, x],
       And(elem(x, sep(A, Lambda(x, elem(x, B)))) ==
           And(elem(x, A), elem(x, B)),
           elem(x, sep(B, Lambda(x, elem(x, A)))) ==
           And(elem(x, A), elem(x, B)))), unknown)

In [5]:
%%file /tmp/sep.smt2

(declare-sort Set 0)
(declare-fun elem (Set Set) Bool)
(declare-fun sep (Set (Array Set Bool)) Set)
(declare-fun knuckledragger_goal () Bool)
(assert (forall ((A Set) (B Set) (x Set))
  (let ((a1 (elem x (sep A (lambda ((x2 Set)) (elem x2 B))))))
    (= a1 (and (elem x A) (elem x B))))))
(assert (forall ((A Set) (B Set) (x Set))
  (let ((a1 (elem x (sep B (lambda ((x1 Set)) (elem x1 A))))))
    (= a1 (and (elem x A) (elem x B))))))
(assert (let ((a1 (forall ((A Set) (B Set) (x Set))
             (let ((a1 (elem x (sep A (lambda ((x1 Set)) (elem x1 B)))))
                   (a2 (elem x (sep B (lambda ((x1 Set)) (elem x1 A))))))
               (and (= a1 (and (elem x A) (elem x B)))
                    (= a2 (and (elem x A) (elem x B))))))))
  (=> knuckledragger_goal (not a1))))

Overwriting /tmp/sep.smt2


In [10]:
import cvc5
slv = cvc5.Solver()

# set that we should print success after each successful command
slv.setOption("print-success", "true")

# construct an input parser associated the solver above
parser = cvc5.InputParser(slv)

input = """
(declare-sort Set 0)
(declare-fun elem (Set Set) Bool)
(declare-fun sep1 (Set (Array Set Bool)) Set)
(declare-fun knuckledragger_goal () Bool)
(assert (forall ((A Set) (B Set) (x Set))
  (let ((a1 (elem x (sep1 A (lambda ((x1 Set)) (elem x1 B))))))
    (= a1 (and (elem x A) (elem x B))))))
(assert (forall ((A Set) (B Set) (x Set))
  (let ((a1 (elem x (sep1 B (lambda ((x1 Set)) (elem x1 A))))))
    (= a1 (and (elem x A) (elem x B))))))
(assert (let ((a1 (forall ((A Set) (B Set) (x Set))
             (let ((a1 (elem x (sep1 A (lambda ((x1 Set)) (elem x1 B)))))
                   (a2 (elem x (sep1 B (lambda ((x1 Set)) (elem x1 A))))))
               (and (= a1 (and (elem x A) (elem x B)))
                    (= a2 (and (elem x A) (elem x B))))))))
  (=> knuckledragger_goal (not a1))))

"""

parser.setStringInput(cvc5.InputLanguage.SMT_LIB_2_6, input, "MyInput")

# get the symbol manager of the parser, used when invoking commands below
sm = parser.getSymbolManager()

# parse commands until finished
while True:
    cmd = parser.nextCommand()
    if cmd.isNull():
        break
    print(f"Executing command {cmd}:")
    # invoke the command on the solver and the symbol manager, print the result
    print(cmd.invoke(slv, sm), end="")

print("Finished parsing commands")

# now, check sat with the solver
r = slv.checkSat()
r

Executing command (declare-sort Set 0):
success
Executing command (declare-fun elem (Set Set) Bool):
success
Executing command (declare-fun sep1 (Set (Array Set Bool)) Set):
success
Executing command (declare-fun knuckledragger_goal () Bool):
success


MyInput:2.2: No set-logic command was given before this point.
MyInput:2.2: cvc5 will make all theories available.
MyInput:2.2: Consider setting a stricter logic for (likely) better performance.
MyInput:2.2: To suppress this warning in the future use (set-logic ALL).


RuntimeError: Expected SMT-LIBv2 qualified indentifier, got `x1` (SYMBOL).

# Universes
Constructible Universe. Is there a finitary or complexity version? How big a formula do you need to build a certain thing.
Von Neumann
V=L


# Alternative Set Theories

# IZF
https://mathstodon.xyz/@rg9119/112713337859199391 more clear than mltt?

Use ileancop + knuckledragger approach?



In [11]:
from prologsolvers import nanocopi
import janus_swi as janus


janus.consult("nanocopi", nanocopi.code)

def axiom(thm : str):
    return thm

def lemma(thm, by=[]):
    query = "&".join(by) + "=>" + thm + ""
    janus.query_once(f"fml2cop({query}, F), leancop_equal(F, F2), prove(F2, _Pf)") # timeout?


lemma("a = a")




PrologError: Syntax error: Operator expected
fml2cop(=
** here **
>a = a, F), leancop_equal(F, F2), prove(F2, _Pf) . 

In [ ]:

"""



# Misc

Kripke Platek ~ rca0 for hereditary finite sets?
admissible sets


https://golem.ph.utexas.edu/category/2013/01/from_set_theory_to_type_theory.html
https://arxiv.org/abs/1808.05204 OMPARING MATERIAL AND STRUCTURAL SET THEORIES - shulman
ECTS material vs structural set theory

BZC bounded zermelo

https://ncatlab.org/nlab/show/SEAR

Elem(A) ==? ArraySort(Set, Q)
We never really say anything about q.
x = Function("Set", Q) gives an element of every set...? oh. That's odd.


The separation axiom _is_ a form of bounded quantifier. Huh.


The tptp version

Axiom of choice
universes
Forcing
Well founded recursion

Open Sets are semdecidable.
What is an open set anyway? We draw a blob without a boundary. Open sets are algerbaically defined as a collection being closed under countable interesection and arbitrary union.
Escardo synthetiuc topology blog posts and paper https://www.cs.bham.ac.uk/~mhe/papers/entcs87.pdf
Thew ideas in Marshall and MarshallB


realizibaility "Associate to a proposition a set of programs in your favorite programing language, an element of that set is a realizer of the proposition"
A proof of two normalizable terms being equal is not needed.
sat(p) has the assignment as proof. sure. Let's say p are ground smt formula 
sat(?p) -> sat(?q) is a prcoesure to turn the satisying assignemt of p into one of q
forall q,   (for any propositional formula q)
exists p, P(p), there is propositional

There are slides on smt somewhere about equisatisfiable and other things.
Is there some example where the intuiitionistic character would come into play

atom = min c^T x st Ax >= b.  Proof = dual vector.

examples and counterexamples.

"The motivation behind the definition of (ϕ → ψ) D given by G¨odel is as follows: from a witness x to the hypothesis ϕD one should be able to obtain a witness u to the conclusion ψ D , such that from a counterexample v to the conclusion one should be able to find a counterexample y to the hypothesis"

forall x, exists y, phi(x,y) -> exists Y forall x phi(x, Y @ x) axiom of choice is an internalized skolemization

blame calculus
exists forall problems are function syunthesis problems. Running dialectica bnackwards?



Comprehension is a relfection principle from logic to objects.  If we weanted to reflect substercutrual loigc , we would not gert sets. We'd get multisets. Similar perhaps if we want a "set theory" that takes some other entity as primitive, like lists or sequences.

Model theory and databases. Query containment using TPTP solvers.


domain theory in python

Can I have a ZF style theory of topology? I say bigunion is fine, but don't have family interesction? Is this what constructive set theory does? What do I need to define big intersection? It seems like I can't have the unrestricted axiom of sepcification if I don't want to allow biginter.
Ic can axiom of specify over any finite set.
My quantifiers have to be bounded.


infinite stes of finite sets
finite sets of infinite sets
inf of ibnf

infinite sets don't really have equality

A two sorted theory of inf sets and finite sets. That's interesting. I guess this is 2nd order arithmetic. finite sets ~ nats. 

Mathematicians version of an idealized programming language

SOme poprewrty of limits. There was a tweet. https://x.com/alpha_convert/status/1762153093355974877?s=20 "preserves limits of sequences" / open neighborhoods. Continuity. Topology is also all about continuity.
"A nice example of a logic program that's monotone but not ω-continuous is ∀ over an infinite set:
foo() :- ∀(X ∈ ℕ) bar(X).
If bar₀ = {0}, bar₁ = {0,1}, barᵢ = {0..i}, the rule fires in the limit ⋃ᵢ barᵢ = ℕ, but not at any finite stage barᵢ, so is not continuous."
The logic programming perspective on this stuff... Hmm.


In fact, any real-valued function f is continuous if and only if it preserves the limits of sequences (though this is not necessarily true when using more general notions of continuity)
Continuity let's you lift limits through functions.    `lim a = lim b -> lim f(a) = lim f(b)` A form of congruence for "limit equality".
https://en.wikipedia.org/wiki/Limit_of_a_sequence

https://en.wikipedia.org/wiki/Interchange_of_limiting_operations
https://en.wikipedia.org/wiki/Iterated_limit#Moore-Osgood_theorem_for_interchanging_limits

https://ncatlab.org/nlab/show/limit#limits_in_analysis 


neg neg is closure
A -> B is interior of union A^c cup B

Putting generators inside anything is tough brcause we don't have equality on generators.
https://www.cs.bham.ac.uk/~mhe/papers/entcs87.pdf

s = Sort("Sierp")

compact ~ finite
We could prehaps replaces Nat with hered finite sets

It could be fun to use some lunatic python introspection to not emit epsilon steps, but instead intterup and continue the. pdb or os signal



In [ ]:
def BW(pts, a , b):
    mid = (a + b) / 2
    high, low = [], []
    for i in pts:
        if i > mid:
            high.append(i)
        else:
            low.append(i)
    if len(high) > len(low):
        return BW(high, mid, b)
    else:
        return BW(low, a, mid)

def intermediate()

How to model these finite sequences.

- a stream of nesting intervals.
- increasing information about bits in a number
- a sequence of rationals


Spec at Ludicrous (but finite) precision. Physically, this is all you could possibly ever need.
Take renormalization for example.

# Lattice

A complete lattice 

In [1]:
from z3 import *

B = BoolSort()
L = DeclareSort("L")
LSet = ArraySort(L, B)

x,y,z = Consts("x y z", L)
A,B,C = Consts("A B C", LSet)
#le = Function("ord", L, L, B) # also maybe use built in?
#le_refl = ForAll([x], le(x,x))
#le_trans = ForAll([x,y,z], Implies(And(le(x,y), le(y,z)), le(x,z))
#le_antisym = ForAll([x,y], Implies(And(le(x,y), le(y,x)), x == y)
le = PartialOrder(L, 0)

join = Function("join", L, L, L)
join_refl = ForAll([x], join(x,x) == x)
join_comm = ForAll([x,y], join(x,y) == join(y,x))
join_assoc = ForAll([x,y,z], join(x, join(y,z)) == join(join(x,y), z))
join_le1 = ForAll([x,y], le(x, join(x,y)))
join_le2 = ForAll([x,y], le(y, join(x,y)))

# least bound.

meet = Function("meet", L, L, L)

# complete lattice
bigjoin = Function("bigjoin", LSet, L)
bigmeet = Function("bigmeet", LSet, L)



def lattice(L, le, join, meet):
    return {
        "Set": L,
        "LSet": ArraySort(L, B),
        "le": le,
        "join": join,
        "meet": meet,
        "join_refl": lemma(ForAll([x], join(x,x) == x), by),
        "join_comm": lemma(ForAll([x,y], join(x,y) == join(y,x)). by),
        "join_assoc": lemma(ForAll([x,y,z], join(x, join(y,z)) == join(join(x,y), z)))
    }


# a true subset type. 
# can I use the free logic ticks?
Monotone = DeclareSort("MonoTone")
monotone_fun = Function("fun", Monotone, ArraySort(L,L))
mono_apply = Function("mono_apply", M, L, L)
mono_apply_def = ForAll([m,x], monoapply(m,x) == monotone_fun(m)[x]) # curry? Hmm.

mono_build = Implies(ForAll([l], le(l,f(l))), Exists([m], monotone_fun(m) == f))

FixedPoints = Lambda([f], Lambda([x], x == f(x)))



# Sequence theory
So I claim, set theory is a reflection of logic, which has commuttivyt and assco rpoerties and abosrption which are actually a pain for a computer



In [ ]:
Tree = DeclareSort("Tree") # Sequence?
child = Function(Tree, Tree,  Ord, B) # all "sets" have a well ordering. Hmm. That's the axiom of choice (?)
finchild = Function(Tree, Tree, Nat, B)

# reflection principle
#formula comprhenesion
def comprehension(T):


def child_separation(T, P): # aka filter.
    return Exists([T2, i], child(T, T1, i) == And(P[T1], child(T2,T1, i) ))

def logic_quote(F):
    # generate tree corresponding to formula.
    pass

Tree = Datatype()
Tree.declare("cons", ("car", Tree), ("cdr", Tree))
Tree.declare("nil")
Tree = Tree.create()



# Moinotone operators

# NBG

Hereditarily finite sets and classes HSet -> Bool


In [6]:
from z3 import *
def lemma(thm, by=[]):
    prove(Implies(And(by), thm))
    return thm
B = BoolSort()
Set = DeclareSort("Set")
PSet = ArraySort(Set,B) # Class?
elem = Function("elem", Set, Set, B)
A,B,C = Consts("A B C", Set)
x,y,z = Consts("x y z", Set)
P,Q = Consts("P Q", PSet)
emp = Const("emp", Set)
emp_def = ForAll([x], elem(x,emp) == False)

ext = ForAll([A,B], ForAll([x], elem(x,A) == elem(x,B)) == (A == B))

upair = Function("upair", Set, Set, Set)
upair_def = ForAll([x,y,z], elem(z,pair(x,y)) == Or(x == z, y == z))

spec = Function("spec", Set, PSet, Set)
spec_def = ForAll([A], ForAll([x], elem(x,spec(A,P)) == And(elem(x,A), P[x])))

# nbg
#class_is_set = ForAll([P], Implies(Exists([x], P[x]), Exists([A], ForAll([x], elem(x,A) == P[x]))))


# curried elem is a way of lifting all sets to class

https://link.springer.com/chapter/10.1007/BFb0014566  Sets in types, types in sets Werner

In [6]:
# aczel encdoing type thoery werner
from z3 import *
A = DeclareSort("A") # indices
PreSet = DeclareSort("PreSet")
Set = Datatype("Set")
# It's just a record. Is there a point to needing adt? Hmm. Maybe there is. It still can't contain itself. But I may have done something bad here.
Set.declare("set", ("sup", ArraySort(A, PreSet))) # Well I could tie the knot with casts.
Set = Set.create()

iso1 = Function("iso", PreSet, Set)
iso2 = Function("iso", Set, PreSet)
x = Const("x", Set)
y = Const("y", PreSet)
iso_def = And(ForAll([x], iso1(iso2(x)) == x), ForAll([y], iso2(iso1(y)) == y))

Set.sup(Lambda([a], iso1(x))) == x # probably ought to be not allowed. But would z3 actually say so? I doubt it.


# Trees

Trees in set theoery are prefix closed sets of sequences.

brouwers "fans" 




In [2]:
def pathset(t):
    res = {()}
    for i, child in enumerate(t):
         res.update((i,) + p for p in pathset(child))
    return res

s = pathset(((),((),())))

def prefixes(s):
    yield from [s[:i] for i in range(len(s)+1)]

def prefix_closure(s):
    return {pref for path in s for pref in prefixes(path)}

s == prefix_closure(s)

True

In [ ]:
def konig(t):
    len(t)

def zorn():

def wellfounded(s):

Konig's lemma
Bolzano weirsruass

The fan theorem https://coq.inria.fr/doc/v8.10/stdlib/Coq.Logic.WeakFan.html


omega-cpo using ordinals?

zorn's lemma 
https://gowers.wordpress.com/2008/08/12/how-to-use-zorns-lemma/

https://coq.inria.fr/doc/v8.10/stdlib/Coq.Logic.ClassicalChoice.html
forall A B : Type (R : A -> B -> Prop), forall x : a, exists y : B, R x y -> exists f : A -> B, forall x : A, R x (f x)
 
def konig(tree):
    




# Nominal Sets
 frozenset + atoms. Finite nominal sets.
Equality of sets is modulo alpha

Frankel Mostowski https://ncatlab.org/nlab/show/Fraenkel-Mostowski+model

We need to search for a permutation. The fact its a permutation doesn't seem that relevant.
Could compile to SAT, asp.
Sort of another example of distillation of AC matching problems. The same sort of permutative search.

Any well defined function should be permutation invariant.

I could perhaps do this by _not_ hash consing the empty set, but hash consing everything else.


@dataclass(frozen=True)
class Atom():
    name:str
    def __eq__(self, perm=None):

class NominalSet():
    val:frozenset[Union[NominalSet,Atom]]
    def __eq__(self, perm=None):
        if perm == None:
            perm = []
        if len(self) != len(other):
            return False
        if len(self) > len(other):
            x, y = other,self
        for p in permutes(x):
            try:
                extend(perm, zip(p,y))
    def fresh(self):
        # return atom fresh in this set
    def fresh(self,a):
        return a not in self.fresh()
    def fvs(self):

    def exponent(self, other):
        # prune anything not equivarant?
    def power()
        


infinite decidable sets...? finite support somehow saves us.



class MonoSet():
    def __pow__(x,y):
        # prune any non monotonic functions?


class ParaSet():
finite parametricity. What if I inferred/checked a type for a dictionary / list of pairs.
partial funs vs funs vs relations
[(a,a) for a in range(3)] + [(a,a) for a in "abcd"]  
Information flow in dictionaries

A parametric type is a static guarantee of good void pointer usage. It is also a prolog vbariable that stays un-unified to anything ground. It is also futures that can stay unforced (ignoring `seq : a -> a` I think this is true in haskell.). 
seq != id
finitary haskell as python dicts







Is "atom" distinct from urelement?
atoms are non equal empty sets. is one encoding
axioms are permutation invariant.
Used toshow axiom of choice is independent in this theroy

In [ ]:
from z3 import *
Atom = DeclareSort("Atom")
Set = 



In [2]:

def finany(A, P):
    assert isinstance(A, frozenset)
    return any(P(a) for a in A)



# iterators as sets

# don't output duplicates
def infset(lst):
    seen = set()
    for item in lst:
        if item not in seen:
            yield item
            seen.add(item)
        else:
            yield None # This gives us the chance to interleave. Hmm. Is this also what the generator ending throws though?

# monotonic set. From a generator, we make increasing sequence of sets
def monoset(gen):
    s = set()
    for item in gen:
        s.add(item)
        yield frozenset(s)

# we can convert a monoset back into a generator
def from_monoset(S):
    sprev = set()
    for s in S:
        for x in s - sprev:
            yield x
        sprev = s

def inter_mono(A,B):
    for a,b in zip(A,B):
        yield a & b

def union_mono(A,B):
    for a,b in zip(A,B):
        yield a | b

def bigunion_mono(C):
    # diagonalize
    cursets = []
    for c in C:
        cursets.append(c)
        yield frozenset.union(*cursets)


#https://plato.stanford.edu/entries/set-theory-constructive/axioms-CZF-IZF.html
# separation schema ( I callled in specification earlier)
# I was not very specific about the nature of P. It was assume it was a decidable predicate i.e. a terminating python function that returns
# true or false
        
def separate(A, P):
    pass


# Is this wrong? 2 omega.
def bad_union(A,B):
    for a in A:
        yield a
    for b in B:
        yield b

# int -> elem fuinctions ---> ordinal -> elem functions
# omega  ->. what if I packed in the limit in there (?). Could I make a compositional calculus?
        
def from_fun(f):
    for i in integers():
        yield f(i)
def to_fun(gen):
    def res(i):
        for j, x in enumerate(gen):
            if j == i:
                return x
    return res

def intersect(A,B):
    # interleave and keep seen sets.
    # emit only once we've seen something come out of both.
    # a _stream_ of sets we can't do this. But for a stream of sets we can do the union.
    # This is the open set principle.
    pass
def bigunion(C):
    # doable
    pass
def bigintersect(C):
    raise ValueError("Can't be implemented except for finite sets.")
    # or we could just implement it and hope for the best
    

# itertools.count
def integers():
    i = 0
    while True:
        yield i
        i += 1
Z = infset(integers)

import itertools


# https://docs.python.org/3/library/fractions.html
from fractions import Fraction
import math
# but even better than a taylor series is a guarantee of accurtacy
def sin(x):
    ans = Fraction(0,1)
    n = 1
    while True:
        ans += - (-1)**n * x**n / math.factorial(n)
        yield ans
        n += 2



class Real():
    def __init__(self):
    def __add__(self):
    def __mul__(self):
    def __sub__(self):
    def __div__(self):
    def __neg__(self):
    def __abs__(self):
    def __pow__(self):
    def __eq__(self):
        # Nope.
        raise Exception("We can never know when reals are equal.")
    def __lt__(self):

class Eudoxus():
    # stream of nearly linear integers.
    # converting between
    pass


IndentationError: expected an indented block after function definition on line 48 (2631604627.py, line 49)

What about `computable(S)` as a predicate. Comprehension over delta 0? And because Peano or whatever isn't strong enough to prove termination of everything....
Definitions make things trickier. Need to track propertiers of definitions somehow. foo(x) = exists z, yada. doesn't mean that foo(x) /\ x > 0 is now quantifier free. 
Knowing a termination measure is enough to turn something into delta_0? You 

# Domain Theory
Winskel
Abramsky
Scott papers

https://ncatlab.org/nlab/show/domain+theory

Smyth, M., Topology, in: S. Abramsky, D. Gabbay and T. Maibaum,
editors, Handbook of Logic in Computer Science, Oxford science publications 1, Clarendon Press, Oxford, 1992 pp. 641–761

cpo vs lattice

Could I close a chain by solving rational equations?


In [ ]:
# The metric topology over the reals https://mathworld.wolfram.com/MetricTopology.html
# We represent open sets by a stream of open intervals/balls contained enitrley within it such that there is eventually one that
# has any point in the set 
# This stream represents the 
def R():
        for i in range(n):
    return (-Fraction(n), Fraction(n))

# use arb?

# a set of everything below the sqrt(n). A cut.
def less_sqrt(x):
    for i in range(n):
        return (-Fraction(n), something)


# can compactify R to include -inf, inf.
    
# a compact set offers also a different interface

# lindelof https://en.wikipedia.org/wiki/Lindel%C3%B6f_space Countable subcovers
    # evidently I am probably only disucssing lindelogf spaces.


# product topology. preimage of projections. Cylinders.

Relationship to convexity?
ConvexSets.jl.  Separation.
Convex is a nice stand in for certain notions of tractable
Huasdorff separation


In [ ]:
class SymReal():
    gen: iter
    expr : Z3Expr

    def __eq__(self, other):
        for i, (x,y) enumerate(zip(self.gen,m other.gen)
            if separate(x,y):
                return False
            # or twee or whatever.
            # run a equational prover and bounds separater concurrently. See who wins.
            if z3.prove(self.expr == other.expr, timeout=i) == unsat:
                return True
            if unknown:
                continue
            if sat
                return False



Autodiffing egraphs. Exact real graph + egraph.

In [ ]:
# according to myhill powerset is gonna be tough.
# but exponentiation is easier

def exp(A,B):
    pass






async python


In [2]:
import asyncio

async def foo():
    await asyncio.sleep(1)
    print("foo")

asyncio.run(foo())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
# actually packing the limit in there.
def recip_n(i):
    if finite(i):
        return 1/i
    else:
        return 0

# finding a limit becomes completing something that only works for finite to something that also handles infinite case

# limit interchange.
# lim_x lim_y  --> lim_y lim_x
def twolim(i):
    if i < omega:
        return f(i,0)
    if omega < i < 2 omega
        return f(omega, i)
    
    

In [ ]:
%%file /tmp/systemt.p

thf( k , type, ) % constant
thf( recurs, type, )
thf( )

k(S,T) = S
s(R,S,T) = r(T) @ s(T)


r(F,G,0) = F
r(F,G,succ(N)) = G @ N @ r(F,G,N)